In [74]:
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import scipy.stats
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('ggplot')
import os
if not os.path.exists("images"):
    os.mkdir("images")
import kaleido 
import plotly.graph_objects as go
import math
from scipy import stats
import matplotlib.pyplot as plt
import math
from scipy import stats
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import ensemble

In [144]:
housing_pp = pd.read_csv('housing_pp.csv', index_col=0)
housing_pp.shape

(2579, 81)

### Split df into independent and dependent variables 

In [260]:
y = housing_pp.SalePrice
X = housing_pp.drop(['SalePrice', 'PID'], axis =1).copy()

In [261]:
y

0       126000
1       139500
2       124900
3       114000
4       227000
         ...  
2575    121000
2576    139600
2577    145000
2578    217500
2579    215000
Name: SalePrice, Length: 2579, dtype: int64

In [262]:
X

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,856,30,RL,62.472926,7890,Pave,0.0,1,4,4,...,166,0,0.0,NaN,NaN,0,3,2010,WD,Normal
1,1049,120,RL,42.000000,4235,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,2,2009,WD,Normal
2,1001,30,C (all),60.000000,6060,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2007,WD,Normal
3,1039,70,RL,80.000000,8146,Pave,0.0,1,4,4,...,111,0,0.0,NaN,NaN,0,5,2009,WD,Normal
4,1665,60,RL,70.000000,8400,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,952,30,RL,70.105866,8854,Pave,0.0,1,4,4,...,40,0,0.0,NaN,NaN,0,5,2009,WD,Normal
2576,1733,20,RL,108.318076,13680,Pave,0.0,2,4,4,...,0,0,0.0,NaN,NaN,0,6,2009,WD,Normal
2577,2002,90,RH,82.000000,6270,Pave,0.0,1,2,4,...,0,0,0.0,NaN,NaN,0,8,2007,WD,Normal
2578,1842,60,RL,69.884162,8826,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,7,2007,WD,Normal


### Use column selector to split data into categorical and numerical variables 
Double check that all columns have been included

In [79]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

In [80]:
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [81]:
len(numerical_columns) + len(categorical_columns) + 2
# +2 for the two dropped columns 

81

### Dummify categorical variables and standardize numerical 
This is just for lasso to start 
This will need to be modified based on the preprocessing for different types of regression

In [248]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", drop= 'first')
numerical_preprocessor = StandardScaler()

In [249]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    [
        ("standard_scaler", numerical_preprocessor, numerical_columns),
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
    ]
)

,GrLivArea,MSSubClass,LotFrontage,LotArea,Alley,LotShape,LandContour,Utilities,LandSlope,OverallQual,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold
0,856,30,62.472926,7890,0.0,1,4,4,3,6,...,0,0,0,0,166,0,0.0,0,3,2010
1,1049,120,42.000000,4235,0.0,1,4,4,3,5,...,0,105,0,0,0,0,0.0,0,2,2009
2,1001,30,60.000000,6060,0.0,1,4,4,3,5,...,154,0,42,86,0,0,0.0,0,11,2007
3,1039,70,80.000000,8146,0.0,1,4,4,3,4,...,0,0,168,0,111,0,0.0,0,5,2009
4,1665,60,70.000000,8400,0.0,1,4,4,3,8,...,0,45,0,0,0,0,0.0,0,11,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,952,30,70.105866,8854,0.0,1,4,4,3,6,...,0,98,0,0,40,0,0.0,0,5,2009
2576,1733,20,108.318076,13680,0.0,2,4,4,3,3,...,0,0,0,0,0,0,0.0,0,6,2009
2577,2002,90,82.000000,6270,0.0,1,2,4,3,5,...,0,0,0,0,0,0,0.0,0,8,2007
2578,1842,60,69.884162,8826,0.0,1,4,4,3,7,...,193,96,0,0,0,0,0.0,0,7,2007


In [84]:
ols = linear_model.LinearRegression()
ridge = linear_model.Ridge()
lasso = linear_model.Lasso()
net = linear_model.ElasticNet()
sgd = linear_model.SGDClassifier
tree = DecisionTreeRegressor

In [250]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

In [251]:
from sklearn.pipeline import make_pipeline

lasso_pipe = make_pipeline(preprocessor, linear_model.Lasso(alpha =10))

In [252]:
import warnings
warnings.filterwarnings('ignore')
lasso_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  ['GrLivArea', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'Alley', 'LotShape',
                                                   'LandContour', 'Utilities',
                                                   'LandSlope', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'ExterQual', 'ExterCond',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinT...
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['MSZoning', 'Street',
                                                   'LotConfig', 'Neighborhood',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'Exterior1st', 'Exterior2nd',
                                                   'MasVnrType', 'Foundation',
                                                   'Heating', 'CentralAir',
                                                   'Electrical', 'GarageType',
                                                   'Fence', 'MiscFeature',
                                                   'SaleType',
                                                   'SaleCondition'])])),
                ('lasso', Lasso(alpha=10))])

In [253]:
lasso_pipe.score(X_train, y_train)

0.9238111256165021

In [254]:
lasso_pipe.score(X_test, y_test)

0.911580269642419

### Check to make sure we are getting the same results without the pipe
**** The categorical variables need to be transformed when they are all together otherwise we might end up getting values that do not exist in training but do exist in testing 

In [263]:
X_ = X
X_

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,856,30,RL,62.472926,7890,Pave,0.0,1,4,4,...,166,0,0.0,NaN,NaN,0,3,2010,WD,Normal
1,1049,120,RL,42.000000,4235,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,2,2009,WD,Normal
2,1001,30,C (all),60.000000,6060,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2007,WD,Normal
3,1039,70,RL,80.000000,8146,Pave,0.0,1,4,4,...,111,0,0.0,NaN,NaN,0,5,2009,WD,Normal
4,1665,60,RL,70.000000,8400,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,952,30,RL,70.105866,8854,Pave,0.0,1,4,4,...,40,0,0.0,NaN,NaN,0,5,2009,WD,Normal
2576,1733,20,RL,108.318076,13680,Pave,0.0,2,4,4,...,0,0,0.0,NaN,NaN,0,6,2009,WD,Normal
2577,2002,90,RH,82.000000,6270,Pave,0.0,1,2,4,...,0,0,0.0,NaN,NaN,0,8,2007,WD,Normal
2578,1842,60,RL,69.884162,8826,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,7,2007,WD,Normal


In [256]:
#X_[numerical_columns] = StandardScaler().fit_transform(X_[numerical_columns])
#X_

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,-1.289220,-0.647048,RL,-0.228004,-0.274310,Pave,-0.248037,-0.703943,0.294819,0.027859,...,2.638313,-0.054802,-0.056524,NaN,NaN,-0.09775,-1.180936,1.647248,WD,Normal
1,-0.894237,1.455666,RL,-0.647755,-0.724056,Pave,-0.248037,-0.703943,0.294819,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,-1.555552,0.884956,WD,Normal
2,-0.992472,-0.647048,C (all),-0.278706,-0.499491,Pave,-0.248037,-0.703943,0.294819,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,1.815997,-0.639628,WD,Normal
3,-0.914703,0.287491,RL,0.131350,-0.242810,Pave,-0.248037,-0.703943,0.294819,0.027859,...,1.669880,-0.054802,-0.056524,NaN,NaN,-0.09775,-0.431702,0.884956,WD,Normal
4,0.366433,0.053856,RL,-0.073678,-0.211555,Pave,-0.248037,-0.703943,0.294819,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,1.815997,0.884956,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,-1.092752,-0.647048,RL,-0.071507,-0.155691,Pave,-0.248037,-0.703943,0.294819,0.027859,...,0.419721,-0.054802,-0.056524,NaN,NaN,-0.09775,-0.431702,0.884956,WD,Normal
2576,0.505598,-0.880683,RL,0.711948,0.438146,Pave,-0.248037,1.048441,0.294819,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,-0.057086,0.884956,WD,Normal
2577,1.056117,0.754761,RH,0.172355,-0.473651,Pave,-0.248037,-0.703943,-3.153427,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,0.692147,-0.639628,WD,Normal
2578,0.728671,0.053856,RL,-0.076053,-0.159136,Pave,-0.248037,-0.703943,0.294819,0.027859,...,-0.284593,-0.054802,-0.056524,NaN,NaN,-0.09775,0.317531,-0.639628,WD,Normal


In [264]:
ohe = OneHotEncoder(handle_unknown="ignore", drop = 'first')
#ohe.fit(X_[categorical_columns])
#X_ = ohe.transform(X_[categorical_columns])
#ohe.categories_
#ohe.fit(X_[categorical_columns])
#X_

In [230]:
#X_.drop(X_[categorical_columns], axis = 1)
#X_

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,856,30,RL,62.472926,7890,Pave,0.0,1,4,4,...,166,0,0.0,NaN,NaN,0,3,2010,WD,Normal
1,1049,120,RL,42.000000,4235,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,2,2009,WD,Normal
2,1001,30,C (all),60.000000,6060,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2007,WD,Normal
3,1039,70,RL,80.000000,8146,Pave,0.0,1,4,4,...,111,0,0.0,NaN,NaN,0,5,2009,WD,Normal
4,1665,60,RL,70.000000,8400,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,952,30,RL,70.105866,8854,Pave,0.0,1,4,4,...,40,0,0.0,NaN,NaN,0,5,2009,WD,Normal
2576,1733,20,RL,108.318076,13680,Pave,0.0,2,4,4,...,0,0,0.0,NaN,NaN,0,6,2009,WD,Normal
2577,2002,90,RH,82.000000,6270,Pave,0.0,1,2,4,...,0,0,0.0,NaN,NaN,0,8,2007,WD,Normal
2578,1842,60,RL,69.884162,8826,Pave,0.0,1,4,4,...,0,0,0.0,NaN,NaN,0,7,2007,WD,Normal


In [232]:
#a = pd.DataFrame(ohe.transform(X_[categorical_columns]).toarray())

In [236]:
#X_ = pd.concat([X_, a], axis = 1)


In [212]:
#pd.DataFrame(preprocessor.fit_transform(X))

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
0,-1.289220,-0.647048,-0.228004,-0.274310,-0.248037,-0.703943,0.294819,0.027859,0.213295,-0.034027,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.894237,1.455666,-0.647755,-0.724056,-0.248037,-0.703943,0.294819,0.027859,0.213295,-0.765323,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.992472,-0.647048,-0.278706,-0.499491,-0.248037,-0.703943,0.294819,0.027859,0.213295,-0.765323,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.914703,0.287491,0.131350,-0.242810,-0.248037,-0.703943,0.294819,0.027859,0.213295,-1.496619,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.366433,0.053856,-0.073678,-0.211555,-0.248037,-0.703943,0.294819,0.027859,0.213295,1.428565,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574,-1.092752,-0.647048,-0.071507,-0.155691,-0.248037,-0.703943,0.294819,0.027859,0.213295,-0.034027,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2575,0.505598,-0.880683,0.711948,0.438146,-0.248037,1.048441,0.294819,0.027859,0.213295,-2.227916,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2576,1.056117,0.754761,0.172355,-0.473651,-0.248037,-0.703943,-3.153427,0.027859,0.213295,-0.765323,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2577,0.728671,0.053856,-0.076053,-0.159136,-0.248037,-0.703943,0.294819,0.027859,0.213295,0.697269,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Take pre processed data and see if we get the same results 

In [265]:
lasso_test = linear_model.Lasso(alpha =10)

In [267]:
X_train_test = pd.DataFrame(preprocessor.fit_transform(X_train))
X_train_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.478490,2.383561,-0.764995,-0.717498,5.339493,1.065833,0.301524,0.032174,0.217328,0.723774,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,-1.007311,2.383561,-1.044731,-0.995578,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.466551,-0.882001,-0.285449,0.064980,-0.246971,1.065833,0.301524,0.032174,0.217328,0.723774,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.896846,0.051017,0.234060,0.220500,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.747354,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,-0.755970,-0.882001,-0.125600,0.359718,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.747354,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,0.339875,0.051017,-0.605146,0.537608,-0.246971,2.838251,0.301524,0.032174,0.217328,0.723774,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1930,-0.651413,3.083325,-0.105619,0.099130,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.747354,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1931,-0.014013,0.051017,-0.165562,-0.228687,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1932,-0.953022,-0.882001,0.014268,-0.269144,-0.246971,1.065833,0.301524,0.032174,0.217328,-0.747354,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [268]:
X_test_test = pd.DataFrame(preprocessor.transform(X_test))
X_test_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.599134,-0.882001,-0.305430,-0.332803,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.361993,0.051017,-0.025694,0.151129,-0.246971,-0.706584,0.301524,0.032174,0.217328,0.723774,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-1.174201,-0.298865,-0.465279,-0.481541,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-1.077687,-0.882001,-0.145581,-0.207864,-0.246971,1.065833,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.629419,0.284271,-0.065656,0.495367,-0.246971,1.065833,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,0.750063,0.051017,-0.005713,-0.029260,-0.246971,-0.706584,0.301524,0.032174,0.217328,1.459337,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
641,-0.703692,0.517526,0.294004,0.626494,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
642,-0.808249,-0.882001,0.234060,0.018217,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.747354,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
643,-0.663477,-0.882001,-0.205524,0.359956,-0.246971,-0.706584,0.301524,0.032174,0.217328,-0.011790,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [269]:
lasso_test.fit(X_train_test, y_train)


Lasso(alpha=10)

In [270]:
lasso_test.score(X_train_test, y_train)

0.9238111256165021

In [271]:
lasso_test.score(X_test_test, y_test)

0.911580269642419

### Now try with CV

In [72]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(lasso_pipe, X, y, cv=5)
cv_results

{'fit_time': array([0.51965809, 0.62033057, 0.52464223, 0.59902668, 0.5644331 ]),
 'score_time': array([0.01718235, 0.01700473, 0.02583241, 0.01800013, 0.01732898]),
 'test_score': array([0.84987588, 0.91536914, 0.90282227, 0.91898491, 0.90962421])}